# DATA MANIPULATION: FEATURES

## 8.1 Feature Extraction

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Python Spark Feature Extraction")
    .config("spark.some.config.option", "some-value")
    .getOrCreate()
)


25/08/31 11:39:10 WARN Utils: Your hostname, gogeon-uui-noteubug.local resolves to a loopback address: 127.0.0.1; using 172.29.56.230 instead (on interface en0)
25/08/31 11:39:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/31 11:39:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/31 11:39:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### TF-IDF

Spark에서는 term frequency vector를 생성하는 방법으로 크게 두가지 1) `HashingTF (줄여서 HTF)`, 2)`CountVectorizer (CV)`가 있다. 두 개의 결과물은 동일하나, 아래의 차이점이 있다.

1. `reversible`(CV) vs `irreversible` (HTF)  
HTF는 각 문장의 토큰들을 hash를 통해 index로 변환 후에 count를 한다. 해시의 특성상 원래의 입력값으로 되돌릴수 없다. 반대로 CV는 모델에 인덱스의 원본 단어를 저장하고 있으므로, reversible하다.

2. memory & computational overhead  
CV는 모든 문장에서 사용되는 토큰들을 수집한 후에, # documents X # tokens의 TF dense vector를 return 한다. 1) 모든 문장을 돌아야 하고, 2) 모든 문장 X 모든 토큰의 TF를 반환하므로 비효율적이다.
HTF는 다른 문장에 dependency없이 각각의 token을 hash 하므로 시간/공간 효율적이다.

3. hashing has dependency from vector size, hashing function, and documents.  
hashing은 다른 토큰이 같은 해시값으로 매핑되는 해시 충돌이 일어날수 있다. 따라서 해시의 버킷 사이즈나 해시 함수등에 영향을 받는다.
4. a source of the information loss  
CV에선 infrequent tokens은 제거하지만, HTF에선 해시 충돌등으로 인해 infrequent token도 다른 토큰으로 병합될수 있다. 사용 환경에 따라서 적절히 알고리즘을 선택할 필요가 있다.

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sentenceData = spark.createDataFrame(
    [(0, "Python python Spark Spark"), (1, "Python SQL")], ["document", "sentence"]
)
sentenceData.show()

+--------+--------------------+
|document|            sentence|
+--------+--------------------+
|       0|Python python Spa...|
|       1|          Python SQL|
+--------+--------------------+



1. CountVectorizer

In [10]:
# Count Vectorizer
from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, Tokenizer
import numpy as np

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
vectorizer = CountVectorizer(inputCol=tokenizer.getOutputCol(), outputCol="features")
idf = IDF(inputCol=vectorizer.getOutputCol(), outputCol="tfidf")
# 토크나이저 -> 벡터라이저 -> IDF 순으로 순차적으로 계산
pipeline = Pipeline(stages=[tokenizer, vectorizer, idf])

model = pipeline.fit(sentenceData)
total_counts = (
    model.transform(sentenceData)
    .select("features")
    .rdd.map(lambda row: row["features"].toArray())
    .reduce(lambda a, b: [a[i] + b[i] for i in range(len(b))])
)

vocabList = model.stages[1].vocabulary
d = {"vocabList": vocabList, "total_counts": total_counts}

# Term frequency
spark.createDataFrame(np.array(list(d.values())).T.tolist(), list(d.keys())).show()

+---------+------------+
|vocabList|total_counts|
+---------+------------+
|   python|         3.0|
|    spark|         2.0|
|      sql|         1.0|
+---------+------------+



vectorizer와 idf 이후 결과는 메모리 효율성을 위해 아래와 같은 형식으로 데이터가 담긴다.
1. 벡터 크기 (int)
2. 비zero값의 인덱스 (list of int)
3. 해당 인덱스의 실제 TF/TF-IDF 값 (list of float)

In [11]:
# 각 문장별 tf-idf 벡터 계산
result = model.transform(sentenceData)
result.show(truncate=False)

+--------+-------------------------+------------------------------+-------------------+----------------------------------+
|document|sentence                 |words                         |features           |tfidf                             |
+--------+-------------------------+------------------------------+-------------------+----------------------------------+
|0       |Python python Spark Spark|[python, python, spark, spark]|(3,[0,1],[2.0,2.0])|(3,[0,1],[0.0,0.8109302162163288])|
|1       |Python SQL               |[python, sql]                 |(3,[0,2],[1.0,1.0])|(3,[0,2],[0.0,0.4054651081081644])|
+--------+-------------------------+------------------------------+-------------------+----------------------------------+



In [15]:
# 원본 termidx를 term word로 변환
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf


def termsIdx2Term(vocabulary):
    def termsIdx2Term(termIndices):
        return [vocabulary[int(index)] for index in termIndices]

    return udf(termsIdx2Term, ArrayType(StringType()))


vectorizerModel = model.stages[1]
vocabList = vectorizerModel.vocabulary
vocabList

['python', 'spark', 'sql']

In [17]:
# 위의 결과를 기반으로 전체 doc X token 행렬을 만든다.

from pyspark.sql.functions import udf
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DoubleType, IntegerType, ArrayType


indices_udf = udf(lambda vector: vector.indices.tolist(), ArrayType(IntegerType()))
tfidf_udf = udf(lambda vector: vector.toArray().tolist(), ArrayType(DoubleType()))

result.select("document", "sentence", "tfidf").withColumn(
    "indices", indices_udf(F.col("tfidf"))
).withColumn("tfidf", tfidf_udf(F.col("tfidf"))).withColumn(
    "Terms", F.size(F.col("indices"))
).withColumn("Terms", termsIdx2Term(vocabList)(F.col("indices"))).show(truncate=False)


+--------+-------------------------+------------------------------+-------+---------------+
|document|sentence                 |tfidf                         |indices|Terms          |
+--------+-------------------------+------------------------------+-------+---------------+
|0       |Python python Spark Spark|[0.0, 0.8109302162163288, 0.0]|[0, 1] |[python, spark]|
|1       |Python SQL               |[0.0, 0.0, 0.4054651081081644]|[0, 2] |[python, sql]  |
+--------+-------------------------+------------------------------+-------+---------------+



2. HashingTF

In [28]:
# Hashing에서 기본으로 사용되는 알고리즘은 murmurhash 3이다.
# 해시 충돌로 인해 다른 단어가 같은 토큰으로 들어갈 수 있다.
vectorizer = HashingTF(inputCol="words", outputCol="features", numFeatures=2000)

pipeline = Pipeline(stages=[tokenizer, vectorizer, idf])

model = pipeline.fit(sentenceData)
result = model.transform(sentenceData)
result.show(truncate=False)


+--------+-------------------------+------------------------------+----------------------------+-------------------------------------------+
|document|sentence                 |words                         |features                    |tfidf                                      |
+--------+-------------------------+------------------------------+----------------------------+-------------------------------------------+
|0       |Python python Spark Spark|[python, python, spark, spark]|(2000,[1286,1709],[2.0,2.0])|(2000,[1286,1709],[0.8109302162163288,0.0])|
|1       |Python SQL               |[python, sql]                 |(2000,[52,1709],[1.0,1.0])  |(2000,[52,1709],[0.4054651081081644,0.0])  |
+--------+-------------------------+------------------------------+----------------------------+-------------------------------------------+



In [ ]:
result.select("document", "sentence", "tfidf").withColumn(
    "indices", indices_udf(F.col("tfidf"))
).withColumn("tfidf", tfidf_udf(F.col("tfidf"))).withColumn(
    "Terms", F.size(F.col("indices"))
).show(truncate=False)

# HashingTF는 원래의 word로 역변환이 불가능하다.
# tf-idf 벡터의 dimension은 hash 함수의 bucket 사이즈에 의존한다.
# .withColumn("Terms", termsIdx2Term(vocabList)(F.col("indices"))).show(truncate=False)


+--------+-------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------